In [1]:
import pandas as pd

In [33]:
import batch_variables2 as v

In [205]:
# INFO = pd.read_csv('../../AMAIA/GetPhoneInfo_batch_sample_info.csv')
connect_info = pd.read_csv('connected_rpv_getphone.csv')
disconnect_info = pd.read_csv('disconnected_rpv_getphone.csv')

In [191]:
wslive_info = INFO[INFO['COMMENTS'].isin(['COMPLETE','NOT IN SERVICE'])]

In [219]:
merged.groupby('COMMENTS').count()

,phone,test_type,Input/Output Delimiter,ProviderNameOut,ProviderCityOut,ProviderStateOut,ProviderLatitudeOut,ProviderLongitudeOut,LineTypeOut,ContactNameOut,...,OFFICE_ADDRESS_LINE_2,OFFICE_ADDRESS_CITY,OFFICE_ADDRESS_STATE,OFFICE_ADDRESS_ZIP,OFFICE_ADDRESS_VERIFIED_UPDATED,OFFICE_TELEPHONE,WSLIVE_FILE_DT,SPECIALTY,pred_recall,0
COMMENTS,,,,,,,,,,,,,,,,,,,,,
COMPLETE,347,347,347,347,347,347,347,347,347,345,...,347,347,347,347,347,347,347,347,347,347
NOT IN SERVICE,492,492,492,484,483,483,480,480,483,452,...,490,491,491,491,492,492,492,492,492,492


In [213]:
output = pd.read_csv('get_phone_batch_output.csv')
output4 = pd.read_csv('old_disconnected.csv')
# output2 = pd.read_csv('get_phone_batch_output.csv')

In [209]:
merged4 = pd.merge(output4, disconnect_info, on='OFFICE_TELEPHONE')

In [217]:
# rpv = pd.read_csv('../RPV_archive.csv')
goopy = pd.merge(merged, rpv, on = 'phone')
goopy.to_csv('infinite_csvs.csv')

In [210]:
# merged2.to_csv('output_and_wslive.csv', index = False)
merged4.to_csv('disconnected_and_wslive.csv', index = False)

In [35]:
connect = merged[merged['COMMENTS'].isin(['COMPLETE'])]
connect=connect.fillna('None')

In [36]:
disconnect = merged[merged['COMMENTS'].isin(['NOT IN SERVICE'])]
disconnect=disconnect.fillna('None')

In [88]:
connect_dict = analyze_numbers(connect)
connect.shape

(347, 44)

In [89]:
disconnect_dict = analyze_numbers(disconnect)
disconnect.shape

(492, 44)

In [90]:
working = pd.DataFrame({'Connected':connect_dict, 'Disconnected': disconnect_dict})

In [91]:
working['Connected %']=(working['Connected']/347)*100

In [92]:
working['Disconnected %']=(working['Disconnected']/492)*100

In [93]:
working

,Connected,Disconnected,Connected %,Disconnected %
Address Matches,185,109,53.314121,22.154472
Business Phone,290,341,83.573487,69.308943
City Matches,276,247,79.538905,50.203252
Connected,248,194,71.469741,39.430894
First Name Match,56,81,16.138329,16.463415
High Quality,223,143,64.265130,29.065041
Last Name Match,57,109,16.426513,22.154472
Low Quality,91,252,26.224784,51.219512
Mailable,278,323,80.115274,65.650407
Medium Quality,31,57,8.933718,11.585366


In [56]:
old_results = pd.read_csv('GetPhoneInfoAPIResults.csv')

In [58]:
old_disconnect = INFO[INFO['COMMENTS'].isin(['NOT IN SERVICE'])]
old_connect = INFO[INFO['COMMENTS'].isin(['CONNECTED'])]

In [63]:
output.dtypes

phone                       int64
test_type                  object
Input/Output Delimiter     object
ProviderNameOut            object
ProviderCityOut            object
ProviderStateOut           object
ProviderLatitudeOut       float64
ProviderLongitudeOut      float64
LineTypeOut                object
ContactNameOut             object
ContactAddressOut          object
ContactCityOut             object
ContactStateOut            object
ContactZipOut              object
ContactPhoneType           object
ContactLatitude           float64
ContactLongitude          float64
ContactSICCode             object
ContactSICDesc             object
ContactQualityScore        object
TokensUsed                float64
SMSAddress                 object
MMSAddress                 object
DateFirstSeen             float64
DateOfPorting              object
NoteCodes                  object
NoteDescriptions           object
Errors                     object
dtype: object

In [103]:
def is_healthy(name):
    med_words = ['endocrin','psych',"pharmac", "dr", 'chiropract', 'hlth', 'disease', 'operation', 'psychotherapy', 'vein', 'emergency',  'pediatric', 'surgical', ' dr', 'perioperative', 'psychiatry', 'family practice' ,'cardiovascular', 'vascular', 'ob gyn','urology','obgyn','radiological','dermatology','chiropractic','orthopaedics','diagnostics', 'gastroent','care center', ' m d','diagnostic','ambulance','anesthesiology','radiolog','endocrinology' ,'oncology','primary care','radiology','pathology','pathologists','glaucoma','cancer', 'cardiology', 'neurology', 'family care', 'pulmonary', 'anesthesia', 'orthope', 'urgent care', 'allergy', 'kidney', 'surgery', 'hosp', 'mri', 'throat', 'dentist', 'med', 'clinic', 'health', 'gastroenter','anesthesiologist','patient','physician','surgeon','doctor','hospital', 'md', 'medical', 'pediatrics', 'm.d.']
    healthy = False
    for word in med_words:
        if word in str(name).lower():
            healthy = True
    return(healthy)

In [85]:
def analyze_numbers(dataframe):
    
    
    address_match = 0
    city_match = 0
    zipcode_match = 0
    healthcount = 0
    mailcount = 0
    connectcount = 0
    portcount = 0
    novalidcount = 0
    unknowncount = 0
    disconnectedcount = 0
    voipcount = 0
    wirelesscount = 0
    no_address_count = 0
    no_date_count = 0
    business_count = 0
    residential_count = 0
    low_count = 0
    high_count = 0
    med_count = 0
    first_name_match = 0
    last_name_match = 0 
    workplace_match = 0
    state_match = 0
    no_name = 0
    SIC_count = 0
        


    for row in dataframe.itertuples():
        if str(row.ContactAddressOut) in str(row.OFFICE_ADDRESS_LINE_2) or str(row.OFFICE_ADDRESS_LINE_2) in str(row.ContactAddressOut):
            address_match += 1
        if str(row.OFFICE_ADDRESS_CITY) in str(row.ContactCityOut):
            city_match += 1
        if v.clean_zipcode(row.OFFICE_ADDRESS_ZIP) == v.clean_zipcode(row.ContactZipOut):
            zipcode_match += 1
        
        if row.ContactStateOut == row.OFFICE_ADDRESS_STATE:
            state_match += 1
        
        if str(row.PHYSICIAN_LAST_NAME).lower() in str(row.ContactNameOut).lower():
            last_name_match +=1
        if str(row.PHYSICIAN_FIRST_NAME).lower() in str(row.ContactNameOut).lower():
            first_name_match +=1
        
        if str(row.OFFICE_ADDRESS_LINE_1).lower() in str(row.ContactNameOut).lower():
            workplace_match += 1

        if is_healthy(row.ContactNameOut)==True:    
            healthcount += 1

        if 'IsMailable' in str(row.NoteDescriptions):
            mailcount += 1
        if 'IsConnected' in str(row.NoteDescriptions):
            connectcount += 1
        if 'IsPorted' in str(row.NoteDescriptions):
            portcount += 1
        if 'NotValid' in str(row.NoteDescriptions):
            novalidcount += 1
        if 'IsUnknownContact' in str(row.NoteDescriptions):
            unknowncount += 1
        if 'IsPossibleDisconnected' in str(row.NoteDescriptions):
            disconnectedcount += 1
        if 'IsPossiblePortableVOIP' in str(row.NoteDescriptions):
            voipcount += 1
        if 'IsWireless' in str(row.NoteDescriptions):
            wirelesscount += 1

        if row.ContactAddressOut == 'None':
            no_address_count += 1
        if row.DateOfPorting == 'None':
            no_date_count += 1
        if row.ContactPhoneType == 'BUSINESS':
            business_count += 1
        if row.ContactPhoneType == 'RESIDENTIAL':
            residential_count += 1
        if row.ContactQualityScore == 'LOW':
            low_count += 1
        if row.ContactQualityScore == 'HIGH':
            high_count += 1
        if row.ContactQualityScore == 'MED':
            med_count += 1
        if row.ContactNameOut =='None':
            no_name += 1
        if row.ContactSICDesc == 'OFFICES AND CLINICS OF MEDICAL DOCTORS':
            SIC_count += 1

    new_dict = {
        'Address Matches': address_match,
        'City Matches': city_match,
        'ZipCode Matches': zipcode_match,
        'No Address': no_address_count,
        'No Date': no_date_count,
        'Relevant Name': healthcount,
        'Business Phone': business_count,
        'Residential Phone': residential_count,
        'Low Quality': low_count,
        'Medium Quality': med_count,
        'High Quality': high_count,
        'Mailable': mailcount,
        'Connected': connectcount,
        'Ported': portcount,
        'Not Valid': novalidcount,
        'Unknown Contact': unknowncount,
        'Possibly Disconnected': disconnectedcount,
        'Possibly Portable VOIP': voipcount,
        'Wireless': wirelesscount,
        'First Name Match':first_name_match,
        'Last Name Match':last_name_match,
        'Workplace Match': workplace_match,
        'State Match': state_match, 
        'No Name': no_name,
        'Relevant SIC': SIC_count
        }
        
    return(new_dict)

In [80]:
output.head()

,phone,test_type,Input/Output Delimiter,ProviderNameOut,ProviderCityOut,ProviderStateOut,ProviderLatitudeOut,ProviderLongitudeOut,LineTypeOut,ContactNameOut,...,ContactSICDesc,ContactQualityScore,TokensUsed,SMSAddress,MMSAddress,DateFirstSeen,DateOfPorting,NoteCodes,NoteDescriptions,Errors
0,6233123020,full,|,CENTURYLINK COMM,PHOENIX,ARIZONA,33.5177,-112.0750,LANDLINE,IRONWOOD CANCER & RESEARCH CTR,...,NaN,MED,35.0,NaN,NaN,NaN,10/19/2017,3,IsPorted,NaN
1,4707323585,full,|,TELEPORT COMM AM-GA,ATLANTA,GEORGIA,33.7890,-84.3937,LANDLINE,COBB WELLSTAR,...,NaN,MED,65.0,NaN,NaN,NaN,10/4/2013,"2,3","IsConnected,IsPorted",NaN
2,5594326212,full,|,LEVEL3 TELECOM OF CA,FRESNO,CALIFORNIA,36.7639,-119.7850,LANDLINE,ZUNIGA STEPHEN I NONE,...,OFFICES AND CLINICS OF MEDICAL DOCTORS,HIGH,65.0,NaN,NaN,NaN,2/25/2019,"1,2,3","IsMailable,IsConnected,IsPorted",NaN
3,4143261514,full,|,AMERITECH WISCONSIN,MILWAUKEE,WISCONSIN,43.0461,-87.9558,LANDLINE,ASCENSION WI,...,NaN,MED,65.0,NaN,NaN,NaN,10/3/2003,"2,3","IsConnected,IsPorted",NaN
4,3527428830,full,|,CENTURYLINK,TAVARES,FLORIDA,28.8003,-81.7354,LANDLINE,GLENN E MILES MD CLINICS MEDICAL,...,OFFICES AND CLINICS OF MEDICAL DOCTORS,HIGH,65.0,NaN,NaN,NaN,NaN,"1,2","IsMailable,IsConnected",NaN


In [98]:
connect.groupby('ContactSICDesc').count()
health
medical
physician

,phone,test_type,Input/Output Delimiter,ProviderNameOut,ProviderCityOut,ProviderStateOut,ProviderLatitudeOut,ProviderLongitudeOut,LineTypeOut,ContactNameOut,...,OFFICE_ADDRESS_CITY,OFFICE_ADDRESS_STATE,OFFICE_ADDRESS_ZIP,OFFICE_ADDRESS_VERIFIED_UPDATED,OFFICE_TELEPHONE,WSLIVE_FILE_DT,SPECIALTY,COMMENTS,pred_recall,0
ContactSICDesc,,,,,,,,,,,,,,,,,,,,,
ACCIDENT AND HEALTH INSURANCE CARRIERS,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
"ACCIDENT AND HEALTH INSURANCE CARRIERS,OFFICES OF HEALTH PRACTITIONER,MEDICAL AND HOSPITAL EQUIPMENT,SPEECH SPECIALIST",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
AUDITING SERVICES,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
COLLEGES AND UNIVERSITIES,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
"COLLEGES AND UNIVERSITIES,MEDICAL INSURANCE ASSOCIATIONS",2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
"COLLEGES AND UNIVERSITIES,OFFICES AND CLINICS OF MEDICAL DOCTORS,OFFICES AND CLINICS OF OSTEOPATHIC PHYSICIANS,EXECUTIVE OFFICES",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
ENDOCRINOLOGIST,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
"ENDOCRINOLOGIST,OFFICES AND CLINICS OF MEDICAL DOCTORS",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
FAMILY COUNSELING SERVICES,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [99]:
disconnect.groupby('ContactSICDesc').count()

,phone,test_type,Input/Output Delimiter,ProviderNameOut,ProviderCityOut,ProviderStateOut,ProviderLatitudeOut,ProviderLongitudeOut,LineTypeOut,ContactNameOut,...,OFFICE_ADDRESS_CITY,OFFICE_ADDRESS_STATE,OFFICE_ADDRESS_ZIP,OFFICE_ADDRESS_VERIFIED_UPDATED,OFFICE_TELEPHONE,WSLIVE_FILE_DT,SPECIALTY,COMMENTS,pred_recall,0
ContactSICDesc,,,,,,,,,,,,,,,,,,,,,
BEAUTY SHOPS,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
COLLEGES AND UNIVERSITIES,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
"COLLEGES AND UNIVERSITIES,OFFICES AND CLINICS OF MEDICAL DOCTORS",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
"DETOXIFICATION CENTER, OUTPATIENT",2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
DIETICIAN,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
ENDOCRINOLOGIST,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
"ENDOCRINOLOGIST,MEDICAL CENTERS,PSYCHIATRISTS AND PSYCHOANALYSTS,MEDICAL INSURANCE ASSOCIATIONS,OFFICES AND CLINICS OF MEDICAL DOCTORS,OFFICES OF HEALTH PRACTITIONER",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
"ENDOCRINOLOGIST,OFFICES AND CLINICS OF MEDICAL DOCTORS,OFFICES AND CLINICS OF OSTEOPATHIC PHYSICIANS",1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
EXECUTIVE OFFICES,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [97]:
disconnect.dtypes

phone                                int64
test_type                           object
Input/Output Delimiter              object
ProviderNameOut                     object
ProviderCityOut                     object
ProviderStateOut                    object
ProviderLatitudeOut                 object
ProviderLongitudeOut                object
LineTypeOut                         object
ContactNameOut                      object
ContactAddressOut                   object
ContactCityOut                      object
ContactStateOut                     object
ContactZipOut                       object
ContactPhoneType                    object
ContactLatitude                     object
ContactLongitude                    object
ContactSICCode                      object
ContactSICDesc                      object
ContactQualityScore                 object
TokensUsed                          object
SMSAddress                          object
MMSAddress                          object
DateFirstSe

In [113]:
disconnect=disconnect.fillna('None')
health_count = 0
count = 0
desc_count = 0
for row in disconnect.itertuples():
    count += 1
    desc = row.ContactSICDesc
    if desc != 'None':
        desc_count += 1
        if  is_healthy(desc) == False:
            health_count += 1
            print(desc)
print(health_count/desc_count*100)
print(desc_count/count*100)
print(health_count/count*100)

PUBLICATION PRINTING
MAYORS' OFFICE
SPEECH SPECIALIST
REHABILITATION SERVICES
FAMILY COUNSELING SERVICES
BEAUTY SHOPS
COLLEGES AND UNIVERSITIES
EXECUTIVE OFFICES
COLLEGES AND UNIVERSITIES
DIETICIAN
MEMBERSHIP ORGANIZATIONS, NEC
7.482993197278912
29.878048780487802
2.2357723577235773


In [101]:
endocrinologist
psych


True

In [114]:
addr_translate = {
    'DR': 'DRIVE',
    'PKWY': 'PARKWAY',
    'RD': 'ROAD',
    'ST': 'STREET',
    'STE': 'SUITE',
    'N': 'NORTH',
    'S': 'SOUTH',
    'E': 'EAST',
    'W': 'WEST',
    'LN': 'LANE',
    'CTR': 'CENTER',
    'CT': 'COURT',
    'BLVD': 'BOULEVARD',
    'CIR': 'CIRCLE',
    'HTS': 'HEIGHTS',
    'AVE': 'AVENUE',
    'HWY': 'HIGHWAY',
    'JCT': 'JUNCTION',
    'LK': 'LAKE',
    'MTN': 'MTN',
    'APT': 'APARTMENT',
    'RM': 'ROOM',
    'PL': 'PLACE',
    'PLZ': 'PLACA',
    'RDG': 'RIDGE',
    'SQ': 'SQUARE',
    'STA': 'STATION',
    'TER': 'TERRACE',
    'TRL': 'TRAIL',
    'TPKE': 'TURNPIKE',
    'VLY': 'VALLEY',
    'IS': 'ISLAND'
}


def translate_addr(addr_string):
    addr_string = addr_string.upper()
    new_addr = []
    
    tokens = addr_string.split()
    
    for t in tokens:
        if t in addr_translate:
            new_addr.append(addr_translate[t])
        else:
            new_addr.append(t)
    
    return ' '.join(new_addr)

In [115]:
translate_addr('10819 s drake')

'10819 SOUTH DRAKE'

In [149]:

addr_translate = {
    'DR': 'DRIVE',
    'PKWY': 'PARKWAY',
    'RD': 'ROAD',
    'ST': 'STREET',
    'STE': 'SUITE',
    'N': 'NORTH',
    'S': 'SOUTH',
    'E': 'EAST',
    'W': 'WEST',
    'LN': 'LANE',
    'CTR': 'CENTER',
    'CT': 'COURT',
    'BLVD': 'BOULEVARD',
    'CIR': 'CIRCLE',
    'HTS': 'HEIGHTS',
    'AVE': 'AVENUE',
    'HWY': 'HIGHWAY',
    'JCT': 'JUNCTION',
    'LK': 'LAKE',
    'MTN': 'MTN',
    'APT': 'APARTMENT',
    'RM': 'ROOM',
    'PL': 'PLACE',
    'PLZ': 'PLACA',
    'RDG': 'RIDGE',
    'SQ': 'SQUARE',
    'STA': 'STATION',
    'TER': 'TERRACE',
    'TRL': 'TRAIL',
    'TPKE': 'TURNPIKE',
    'VLY': 'VALLEY',
    'IS': 'ISLAND'
}


def translate_addr(addr_string):
    addr_string = addr_string.upper()
    new_addr = []
    
    tokens = addr_string.split()
    
    for t in tokens:
        if t in addr_translate:
            new_addr.append(addr_translate[t])
        else:
            new_addr.append(t)
    
    return ' '.join(new_addr)


def is_healthy(name):
    healthy = False
    for word in med_words:
        if word in str(name).lower():
            healthy = True   
    return(healthy)

SyntaxError: invalid syntax (<ipython-input-149-89b516ec556c>, line 161)

In [181]:
def variable_breakdown(dataframe):
    
    LIST_OF_DICTS = []
    
    
    for row in dataframe.itertuples():

        address_match = 0
        business = 0
        city_match = 0
        connect = 0
        no_date = 0
        disconnected = 0
        error = 0
        first_name_match = 0
        health = 0
        high = 0
        INF = 0
        landline = 0
        last_name_match = 0 
        low = 0
        mail = 0
        med = 0
        no_address = 0
        no_lat =0
        no_linetype = 0
        no_provider = 0
        no_provider_city = 0
        no_provider_state = 0
        no_SIC = 0
        not_valid = 0
        port = 0
        possible_portable_voip =0
        relevant_SIC =0
        residential = 0
        state_match = 0
        toll_free = 0
        unknown = 0
        unknown_phone_type = 0 
        voip = 0
        wireless_note =0
        wireless_type = 0
        workplace_match = 0
        zipcode_match = 0



        gpi_add = translate_addr(str(row.ContactAddressOut))
        ws_add = translate_addr(str(row.OFFICE_ADDRESS_LINE_2))
        if gpi_add != 'NONE':
            if gpi_add in ws_add or ws_add in gpi_add:
                address_match = 1

        if row.ContactPhoneType == 'BUSINESS':
            business = 1

        if str(row.OFFICE_ADDRESS_CITY) in str(row.ContactCityOut):
            city_match = 1

        if 'IsConnected' in str(row.NoteDescriptions):
            connect = 1

        if row.DateOfPorting == 'None':
            no_date = -1

        if 'IsPossibleDisconnected' in str(row.NoteDescriptions):
            disconnected = -1
    
        if row.Errors == 'GeoPhonePlus2 error: Phone Number is not valid.':
            error = -1
        
        if str(row.PHYSICIAN_FIRST_NAME).lower() in str(row.ContactNameOut).lower():
            first_name_match =1
            
        if is_healthy(row.ContactNameOut) == True:  
            health = 1
        
        if row.ContactQualityScore == 'HIGH':
            high = 1
        
        if 'INF' in str(row.NoteDescriptions):
            INF = -1

        if row.LineTypeOut == 'LANDLINE':
            landline = 1
        
        if str(row.PHYSICIAN_LAST_NAME).lower() in str(row.ContactNameOut).lower():
            last_name_match =1

        if row.ContactQualityScore == 'LOW':
            low = -1

        if 'IsMailable' in str(row.NoteDescriptions):
            mail = 1

        if row.ContactQualityScore == 'MED':
            med = 0.5

        if row.ContactAddressOut == 'None':
            no_address = -1

        if row.ContactLatitude == 'None':
            no_lat = -1

        if row.LineTypeOut == 'None':
            no_linetype = -1

        if row.ProviderNameOut == 'None':
            no_provider = -1
        
        if row.ProviderCityOut == 'None':
            no_provider_city = -1
        
        if row.ProviderStateOut == 'None':
            no_provider_state = -1

        if row.ContactSICDesc == 'None':
            no_SIC = -1

        if 'NotValid' in str(row.NoteDescriptions):
            not_valid = -1   

        if 'IsPorted' in str(row.NoteDescriptions):
            port = 1
        
        if 'IsPossiblePortableVOIP' in str(row.NoteDescriptions):
            possible_portable_voip = -1

        if is_healthy(row.ContactSICDesc) == True:
            relevant_SIC = 1
            
        if row.ContactPhoneType == 'RESIDENTIAL':
            residential = 1
            
        if row.ContactStateOut == row.OFFICE_ADDRESS_STATE:
            state_match = 1

        if 'IsTollFreeNumber' in str(row.NoteDescriptions):
            toll_free = -1

        if 'IsUnknownContact' in str(row.NoteDescriptions):
            unknown = -1

        if row.ContactPhoneType == 'UNKNOWN':
            unknown_phone_type = -1

        if row.LineTypeOut == 'VOIP':
            voip = 1
        
        if 'IsWireless' in str(row.NoteDescriptions):
            wireless_note = -1

        if row.LineTypeOut == 'WIRELESS':
            wireless_type += -1

        if str(row.OFFICE_ADDRESS_LINE_1).lower() in str(row.ContactNameOut).lower():
            workplace_match = 1

        if clean_zipcode(row.OFFICE_ADDRESS_ZIP) == clean_zipcode(row.ContactZipOut):
            zipcode_match = 1
        
        


        new_dict = {
            'OFFICE_TELEPHONE': row.OFFICE_TELEPHONE,
            'Address Match': address_match,
            'Business Phone': business,
            'City Match': city_match,
            'Connected': connect,
            'Disconnected':disconnected,
            'Error':error,
            'First Name Match':first_name_match,
            'High Quality': high,
            'INF': INF,
            'Landline': landline,
            'Last Name Match':last_name_match,
            'Low Quality': low,
            'Mailable': mail,
            'Medium Quality': med,
            'No Address': no_address,
            'No Geocoordinates': no_lat,
            'No Line Type': no_linetype,
            'No Provider': no_provider,
            'No Provider City': no_provider_city,
            'No Provider State': no_provider_state,
            'NO SIC': no_SIC,
            'Not valid': not_valid,
            'No Date': no_date,
            'Ported': port,
            'Possibly Portable VOIP': possible_portable_voip,
            'Relevant Name': health,
            'Relevant SIC': relevant_SIC,
            'Residential Phone': residential,
            'State Match': state_match,
            'Toll free': toll_free,
            'Unknown Contact': unknown,
            'Unknown phone type':unknown_phone_type,
            'Workplace Match': workplace_match,
            'ZipCode Match': zipcode_match,
            'Wireless type': wireless_type,
            'Wireless note': wireless_note,
            'VOIP': voip
            }

        LIST_OF_DICTS.append(new_dict)

    return(LIST_OF_DICTS)

In [152]:
def analyze_numbers(dataframe):
    
    dataframe=dataframe.fillna("None")
    
    address_match = 0
    business = 0
    city_match = 0
    connect = 0
    no_date = 0
    disconnected = 0
    error = 0
    first_name_match = 0
    health = 0
    high = 0
    INF = 0
    landline = 0
    last_name_match = 0 
    low = 0
    mail = 0
    med = 0
    no_address = 0
    no_lat =0
    no_linetype = 0
    no_provider = 0
    no_provider_city = 0
    no_provider_state = 0
    no_SIC = 0
    not_valid = 0
    port = 0
    possible_portable_voip =0
    relevant_SIC =0
    residential = 0
    state_match = 0
    toll_free = 0
    unknown = 0
    unknown_phone_type = 0 
    voip = 0
    wireless_note =0
    wireless_type = 0
    workplace_match = 0
    zipcode_match = 0
    
    for row in dataframe.itertuples():

        gpi_add = translate_addr(str(row.ContactAddressOut))
        ws_add = translate_addr(str(row.OFFICE_ADDRESS_LINE_2))
        if gpi_add != 'NONE':
            if gpi_add in ws_add or ws_add in gpi_add:
                address_match += 1

        if row.ContactPhoneType == 'BUSINESS':
            business += 1

        if str(row.OFFICE_ADDRESS_CITY) in str(row.ContactCityOut):
            city_match += 1

        if 'IsConnected' in str(row.NoteDescriptions):
            connect += 1

        if row.DateOfPorting == 'None':
            no_date += 1

        if 'IsPossibleDisconnected' in str(row.NoteDescriptions):
            disconnected += 1
    
        if row.Errors == 'GeoPhonePlus2 error: Phone Number is not valid.':
            error += 1
        
        if str(row.PHYSICIAN_FIRST_NAME).lower() in str(row.ContactNameOut).lower():
            first_name_match +=1
            
        if is_healthy(row.ContactNameOut) == True:  
            health += 1
        
        if row.ContactQualityScore == 'HIGH':
            high += 1
        
        if 'INF' in str(row.NoteDescriptions):
            INF += 1

        if row.LineTypeOut == 'LANDLINE':
            landline += 1
        
        if str(row.PHYSICIAN_LAST_NAME).lower() in str(row.ContactNameOut).lower():
            last_name_match +=1

        if row.ContactQualityScore == 'LOW':
            low += 1

        if 'IsMailable' in str(row.NoteDescriptions):
            mail += 1

        if row.ContactQualityScore == 'MED':
            med += 1

        if row.ContactAddressOut == 'None':
            no_address += 1

        if row.ContactLatitude == 'None':
            no_lat += 1

        if row.LineTypeOut == 'None':
            no_linetype += 1

        if row.ProviderNameOut == 'None':
            no_provider += 1
        
        if row.ProviderCityOut == 'None':
            no_provider_city += 1
        
        if row.ProviderStateOut == 'None':
            no_provider_state += 1

        if row.ContactSICDesc == 'None':
            no_SIC += 1

        if 'NotValid' in str(row.NoteDescriptions):
            not_valid += 1   

        if 'IsPorted' in str(row.NoteDescriptions):
            port += 1
        
        if 'IsPossiblePortableVOIP' in str(row.NoteDescriptions):
            possible_portable_voip += 1

        if is_healthy(row.ContactSICDesc) == True:
            relevant_SIC += 1
            
        if row.ContactPhoneType == 'RESIDENTIAL':
            residential += 1
            
        if row.ContactStateOut == row.OFFICE_ADDRESS_STATE:
            state_match += 1

        if 'IsTollFreeNumber' in str(row.NoteDescriptions):
            toll_free += 1

        if 'IsUnknownContact' in str(row.NoteDescriptions):
            unknown += 1

        if row.ContactPhoneType == 'UNKNOWN':
            unknown_phone_type += 1

        if row.LineTypeOut == 'VOIP':
            voip += 1
        
        if 'IsWireless' in str(row.NoteDescriptions):
            wireless_note += 1

        if row.LineTypeOut == 'WIRELESS':
            wireless_type += 1

        if str(row.OFFICE_ADDRESS_LINE_1).lower() in str(row.ContactNameOut).lower():
            workplace_match += 1

        if clean_zipcode(row.OFFICE_ADDRESS_ZIP) == clean_zipcode(row.ContactZipOut):
            zipcode_match += 1
        
        


    new_dict = {
        'Address Match': address_match,
        'Business Phone': business,
        'City Match': city_match,
        'Connected': connect,
        'Disconnected':disconnected,
        'Error':error,
        'First Name Match':first_name_match,
        'High Quality': high,
        'INF': INF,
        'Landline': landline,
        'Last Name Match':last_name_match,
        'Low Quality': low,
        'Mailable': mail,
        'Medium Quality': med,
        'No Address': no_address,
        'No Geocoordinates': no_lat,
        'No Line Type': no_linetype,
        'No Provider': no_provider,
        'No Provider City': no_provider_city,
        'No Provider State': no_provider_state,
        'NO SIC': no_SIC,
        'Not valid': not_valid,
        'No Date': no_date,
        'Ported': port,
        'Possibly Portable VOIP': possible_portable_voip,
        'Relevant Name': health,
        'Relevant SIC': relevant_SIC,
        'Residential Phone': residential,
        'State Match': state_match,
        'Toll free': toll_free,
        'Unknown Contact': unknown,
        'Unknown phone type':unknown_phone_type,
        'Workplace Match': workplace_match,
        'ZipCode Match': zipcode_match,
        'Wireless type': wireless_type,
        'Wireless note': wireless_note,
        'VOIP': voip
        }


    return(new_dict)

In [146]:
disconnect_dict = analyze_numbers(disconnect)
connect_dict = analyze_numbers(connect)
analyze_df = pd.DataFrame({'Connected': connect_dict, 'Disconnected':disconnect_dict})

In [186]:
analyze_df['Connected %']=(analyze_df['Connected']/347)*100
analyze_df['Disconnected %']=(analyze_df['Disconnected']/492)*100
analyze_df['100-Disconnected %']=100-analyze_df['Disconnected %']
analyze_df['100-Connected %']=100-analyze_df['Connected %']
analyze_df

,Connected,Disconnected,Connected %,Disconnected %,100-Disconnected %,100-Connected %
Address Match,186,107,53.602305,21.747967,78.252033,46.397695
Business Phone,290,341,83.573487,69.308943,30.691057,16.426513
City Match,276,247,79.538905,50.203252,49.796748,20.461095
Connected,248,194,71.469741,39.430894,60.569106,28.530259
Disconnected,10,109,2.881844,22.154472,77.845528,97.118156
Error,0,8,0.000000,1.626016,98.373984,100.000000
First Name Match,56,81,16.138329,16.463415,83.536585,83.861671
High Quality,223,143,64.265130,29.065041,70.934959,35.734870
INF,10,109,2.881844,22.154472,77.845528,97.118156
Landline,279,417,80.403458,84.756098,15.243902,19.596542


In [125]:
def clean_zipcode(zipcode):
    zipcode=str(zipcode)
    zipcode = zipcode.replace(".0","")
    if len(zipcode)==3:
        zipcode = "00"+zipcode
    if len(zipcode)==4: 
        zipcode = "0"+zipcode
    else:
        zipcode = zipcode[0:5]
    return(zipcode)

In [182]:
connect_variables = variable_breakdown(connect)

In [183]:
conn_df = pd.DataFrame(connect_variables)

In [184]:
disconnect_variables = variable_breakdown(disconnect)
disconn_df = pd.DataFrame(disconnect_variables)

In [185]:
disconn_df.to_csv('batch_dis_variables.csv')
conn_df.to_csv('batch_con_variables.csv')